In [2]:
import osmnx as ox
import requests
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import warnings
import time
import math

In [3]:
%%time
# Webscraping by looking at HTML-table formats. Get de Paris Urban area (Paris Aire Urbaine) 
# by French statistic bureau INSEE via citypopulation.de

city = 'Paris'
url = 'https://citypopulation.de/en/france/paris/admin/'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]
df

Wall time: 1.16 s


,Name,Status,PopulationEstimate2007-01-01,PopulationEstimate2015-01-01,PopulationEstimate2018-01-01,Unnamed: 5
0,Ablon-sur-Seine,Commune,5155,5527,5818,→
1,Ablon-sur-Seine,Commune,5155,5527,5818,→
2,Achères,Commune,19789,21053,21098,→
3,Achères Gare,Statistical Area,139,198,235,→
4,Centre Ville,Statistical Area,2066,2358,2761,→
...,...,...,...,...,...,...
4708,Gambetta-Brossolette-GARE,Statistical Area,4224,4315,4367,→
4709,Gros Bois-Mare Armée-Sablière,Statistical Area,3558,3818,3730,→
4710,Rives de l'Yerres-Tournelles,Statistical Area,4555,4473,4493,→
4711,Taillis-Garenne,Statistical Area,4063,4139,4116,→


In [4]:
%%time
# Formatting, only take communes and municipal arrondissements, no higher or lower level
unités_urbaine = df[(df['Status'] == 'Commune') | (df['Status'] == 'Municipal Arrondissement')]
# Drop duplicates (this is due to communes that consist of one statistical area also getting 'commune' tag)
unités_urbaine = unités_urbaine.drop_duplicates()
# Renaming columns and drop last. Because population information can be useful later.
unités_urbaine.columns = ['name','status','pop2007','pop2015','pop2018','no_name']
print('Population: ',city,sum(unités_urbaine['pop2018']))
unités_urbaine = unités_urbaine.iloc[:,:5]
unités_urbaine

Population:  Paris 10829408
Wall time: 3.99 ms


,name,status,pop2007,pop2015,pop2018
0,Ablon-sur-Seine,Commune,5155,5527,5818
2,Achères,Commune,19789,21053,21098
12,Alfortville,Commune,44116,44410,44287
28,Andilly,Commune,2449,2572,2581
30,Andrésy,Commune,12501,12403,13078
...,...,...,...,...,...
4645,Viry-Châtillon,Commune,31249,30831,30706
4658,Vitry-sur-Seine,Commune,83650,92531,94649
4696,Voisins-le-Bretonneux,Commune,12153,11378,10921
4702,Wissous,Commune,5112,7687,7301


In [5]:
%%time
# Get Paris and the cities included in the Aire Urbaine, get the right 'Marolles-en-Brie' (two present in Ile-de-France)
locals()[city] = list(unités_urbaine[unités_urbaine['name'] != 'Marolles-en-Brie']['name']+', Ile-de-France')
locals()[city].append('Marolles-en-Brie, Val-de-Marne, Ile-de-France')

warnings.filterwarnings('ignore')
# Extract bike networks according to this list
locals()['Bike'+city] = ox.graph_from_place(locals()[city], network_type = "bike", 
                                            retain_all = True, clean_periphery = True)

Wall time: 5min 58s


In [6]:
%%time
# Get the nodes and edges from the graph
nodes, edges = ox.graph_to_gdfs(locals()['Bike'+city])

Wall time: 1min 2s


In [7]:
%%time
# Find cycleway if present
edges['highway'] = np.where(edges['highway'].isin(['cycleway']),'cycleway',edges['highway'])

#edges = edges.iloc[:,0:8]

# 1. Unlist items if needed. First item (as most important) counts
for j in range(len(edges.columns)):
    for i in range(len(edges)):
        if isinstance(edges.iloc[i,j], list): # 1
            edges.iloc[i,j] = edges.iloc[i,j][0]
    print(j)
edges

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
Wall time: 5min 46s


osmid lanes             name      highway  \
u           v           key                                                   
360930      2301459230  0      84416296     3   Rue de Tolbiac    secondary   
            361704      0      84416296     3   Rue de Tolbiac    secondary   
360932      253316556   0       3599327     2   Rue de Tolbiac    secondary   
            5487025239  0     157910468     3   Rue de Tolbiac    secondary   
            252659093   0     309765258     5  Avenue d'Italie      primary   
...                                 ...   ...              ...          ...   
10097908414 10097908413 0    1103511577   NaN              NaN      service   
10097908415 10097948620 0    1103511580   NaN      Chemin Pavé  residential   
                        1    1103511578   NaN              NaN      service   
10097948620 858034594   0    1103511580   NaN      Chemin Pavé  residential   
10097948621 858034579   0      72180571   NaN              NaN  residential   

                            maxspeed  oneway   length  \
u           v           key                             
360930      2301459230  0         30   False   26.827   
            361704      0         30   False  114.399   
360932      253316556   0         30   False  100.372   
            5487025239  0         30   False   90.883   
            252659093   0         30   False   44.347   
...                              ...     ...      ...   
10097908414 10097908413 0        NaN   False    7.308   
10097908415 10097948620 0        NaN    True   16.719   
                        1        NaN    True   22.695   
10097948620 858034594   0        NaN    True   37.909   
10097948621 858034579   0        NaN   False   62.118   

                                                                      geometry  \
u           v           key                                                      
360930      2301459230  0      LINESTRING (2.36150 48.82635, 2.36186 48.82639)   
            361704      0    LINESTRING (2.36150 48.82635, 2.36074 48.82624...   
360932      253316556   0    LINESTRING (2.35726 48.82618, 2.35747 48.82618...   
            5487025239  0    LINESTRING (2.35726 48.82618, 2.35707 48.82618...   
            252659093   0    LINESTRING (2.35726 48.82618, 2.35731 48.82605...   
...                                                                        ...   
10097908414 10097908413 0      LINESTRING (2.06505 48.86505, 2.06514 48.86507)   
10097908415 10097948620 0      LINESTRING (2.06590 48.86538, 2.06604 48.86550)   
                        1    LINESTRING (2.06590 48.86538, 2.06588 48.86540...   
10097948620 858034594   0    LINESTRING (2.06604 48.86550, 2.06616 48.86561...   
10097948621 858034579   0      LINESTRING (2.06293 48.86621, 2.06344 48.86576)   

                            width  ref junction tunnel bridge access service  \
u           v           key                                                    
360930      2301459230  0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
            361704      0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
360932      253316556   0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
            5487025239  0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
            252659093   0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
...                           ...  ...      ...    ...    ...    ...     ...   
10097908414 10097908413 0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
10097908415 10097948620 0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
                        1     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
10097948620 858034594   0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
10097948621 858034579   0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   

                            est_width area landuse  
u           v           key                         
360930      2301459230  0         NaN  NaN     NaN

In [8]:
# Removing roads faster than 90 km/h
ed2 = edges[~edges['maxspeed'].isin(['walk','FR:walk','signals'])]
ed2 = pd.concat([ed2[(ed2['maxspeed'].astype(float) > 90) == False],
                 edges[edges['maxspeed'].isin(['walk','FR:walk','signals'])]])

# Make a dedicated "number-based" speed column
speed = ed2['maxspeed']
speed = pd.Series(np.where(speed.isin(['walk','FR:walk']),5,np.where(speed.isin(['signals']),50,speed)), dtype = float)
speed.index = ed2.index

# Transform oneway and lanes to numberwise columns
ed2['cleanspeed'] = speed
ed2['oneway'] = ed2['oneway'].astype(int)
ed2['lanes'] = pd.Series(np.where(ed2['lanes'] =='Allée des petits clos',1,ed2['lanes']),index = ed2.index, dtype =float)
ed2

osmid  lanes                   name  \
u           v          key                                             
360930      2301459230 0      84416296    3.0         Rue de Tolbiac   
            361704     0      84416296    3.0         Rue de Tolbiac   
360932      253316556  0       3599327    2.0         Rue de Tolbiac   
            5487025239 0     157910468    3.0         Rue de Tolbiac   
            252659093  0     309765258    5.0        Avenue d'Italie   
...                                ...    ...                    ...   
10000480189 5520660517 0    1096400097    NaN  Place Raoul Follereau   
465629523   1216749872 0     105665034    NaN       Village des Buis   
1216749872  465629523  0     105665034    NaN       Village des Buis   
2626016775  2626016779 0     256983630    NaN    Clos des Vieux Mûrs   
2626016779  2626016775 0     256983630    NaN    Clos des Vieux Mûrs   

                                highway maxspeed  oneway   length  \
u           v          key                                          
360930      2301459230 0      secondary       30       0   26.827   
            361704     0      secondary       30       0  114.399   
360932      253316556  0      secondary       30       0  100.372   
            5487025239 0      secondary       30       0   90.883   
            252659093  0        primary       30       0   44.347   
...                                 ...      ...     ...      ...   
10000480189 5520660517 0     pedestrian     walk       0   38.586   
465629523   1216749872 0    residential     walk       0  196.587   
1216749872  465629523  0    residential     walk       0  196.587   
2626016775  2626016779 0    residential     walk       0  133.305   
2626016779  2626016775 0    residential     walk       0  133.305   

                                                                     geometry  \
u           v          key                                                      
360930      2301459230 0      LINESTRING (2.36150 48.82635, 2.36186 48.82639)   
            361704     0    LINESTRING (2.36150 48.82635, 2.36074 48.82624...   
360932      253316556  0    LINESTRING (2.35726 48.82618, 2.35747 48.82618...   
            5487025239 0    LINESTRING (2.35726 48.82618, 2.35707 48.82618...   
            252659093  0    LINESTRING (2.35726 48.82618, 2.35731 48.82605...   
...                                                                       ...   
10000480189 5520660517 0    LINESTRING (2.36351 48.87656, 2.36339 48.87661...   
465629523   1216749872 0    LINESTRING (2.06240 48.99380, 2.06149 48.99373...   
1216749872  465629523  0    LINESTRING (2.06022 48.99356, 2.06036 48.99335...   
2626016775  2626016779 0    LINESTRING (2.06254 48.99338, 2.06347 48.99346...   
2626016779  2626016775 0    LINESTRING (2.06432 48.99350, 2.06425 48.99345...   

                           width  ref junction tunnel bridge access service  \
u           v          key                                                    
360930      2301459230 0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
            361704     0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
360932      253316556  0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
            5487025239 0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
            252659093  0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
...                          ...  ...      ...    ...    ...    ...     ...   
10000480189 5520660517 0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
465629523   1216749872 0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
1216749872  465629523  0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
2626016775  2626016779 0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   
2626016779  2626016775 0     NaN  NaN      NaN    NaN    NaN    NaN     NaN   

                           est_width area landuse  cleanspeed  
u           v          key                          

In [27]:
%%time
# Set nodes location to meters (for later filter in R)
pd.set_option('display.float_format', lambda x: '%.1f' % x)
nodes['geom_x_m'] = nodes['geometry'].to_crs(3043).x
nodes['geom_y_m'] = nodes['geometry'].to_crs(3043).y

nodes['osmid_var'] = nodes.index
nodes['osmid_var'] = nodes['osmid_var'].astype('int64')

# Make if path not exists and save the files there.
warnings.filterwarnings('ignore')
if not os.path.exists('D:/EconNet/'+str(city)+'/'):
    os.makedirs('D:/EconNet/'+str(city)+'/')
nodes.to_file('D:/EconNet/'+str(city)+'/Cycling_routes/BikePoints.shp')
ed2.to_file('D:/EconNet/'+str(city)+'/Cycling_routes/BikeRoutes.shp')

Wall time: 1min 14s


In [26]:
nodes.sort_values('osmid')

,y,x,street_count,highway,ref,geometry,geom_x_m,geom_y_m,osmid_var
osmid,,,,,,,,,
125730,48.9,2.4,4,NaN,NaN,POINT (2.41073 48.86352),456781.9,5412451.9,125730
125742,48.9,2.4,4,NaN,NaN,POINT (2.40137 48.85185),456085.3,5411159.8,125742
125747,48.9,2.4,3,NaN,NaN,POINT (2.41466 48.85426),457062.4,5411420.0,125747
125760,48.8,2.4,3,NaN,NaN,POINT (2.40126 48.82951),456057.3,5408676.5,125760
360930,48.8,2.4,3,NaN,NaN,POINT (2.36150 48.82635),453136.4,5408348.3,360930
...,...,...,...,...,...,...,...,...,...
10098476384,48.6,2.4,1,NaN,NaN,POINT (2.43377 48.63272),458281.0,5386782.5,10098476384
10098476387,48.6,2.4,3,NaN,NaN,POINT (2.43404 48.63286),458301.1,5386798.1,10098476387
10098476396,48.6,2.4,1,NaN,NaN,POINT (2.43361 48.63312),458269.4,5386828.1,10098476396
